# Customer Claims Hana Connection


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc("font",size=14)
sns.set(style="white")
sns.set(style="whitegrid",color_codes=True)
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import pyodbc 
conn = None
cursor = None
conn = pyodbc.connect (
                        DSN = 'HANA PROD',
                        uid = 'MART_RDR_2',
                        pwd = 'Re@d3r13')
#cursor = conn.cursor()
#cursor.execute("select current_date from dummy")
#for row in cursor.fetchall():
#print(row)
df = pd.read_sql_query("""select 
	Claim.CLAIM_NUM 
	,Claim.INVENTORY_STYLE_CD 
	,Claim.INVENTORY_SIZE_CD 
	,Claim.INVENTORY_BACKING_CD 
	,Claim.INVENTORY_COLOR_CD 
	,Claim.DIM_DIVISION_CUSTOMER_GK 
	,Claim.F1ROLL
	,Claim.F1PROL
	,Claim.F1GROL
	,Claim.F1DLOT
	,Claim.Claim_Status_Type_Cd
	,sum(claim) as Claim 
	,sum(Total_Claim) as Total_Claim 
	,sum(Claim_Cnt) as Total_Claim_Cnt 
	,sum(Trx_cnt) as Trx_cnt 
	,sum(Net_Sales) as Net_Sales  
	from  
		(  
			select 
					CLAIM_NUM 
					,INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK 
					,F1ROLL
					,F1PROL
					,F1GROL
					,F1DLOT
					,Claim_Status_Type_Cd
					,sum(TOTAL_CLAIMS_DOLLAR_AMT) as Claim 
			from 
					"MHK_EDW_MART"."FACT_CUSTOMER_CLAIMS" A 
					,"MHK_EDW_MART"."DIM_CLAIM_STATUS" B 
					,"MHK_EDW_MART"."DIM_CLAIM_REASON" C 
					,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" D 
					,"MHK_EDW_MART"."FIP010"  E
			where 
					A.DIM_CLAIM_STATUS_PK=B.DIM_CLAIM_STATUS_PK 
					and A.DIM_CLAIM_REASON_PK = C.DIM_CLAIM_REASON_PK 
					and A.DIM_FINISHED_PRODUCT_MFG_GK = D.DIM_FINISHED_PRODUCT_MFG_GK 
					and QUALITY_RELATED_CD = 'Y' and A.DIM_FINISHED_PRODUCT_MFG_GK >0 
					and A.FINISHED_ROLL_NUM = E.F1ROLL
					and Claim_Status_Type_Cd in ('48','49','50','51','30','35','36','34')
					and A.CLAIM_STATUS_DATE_PK >20170101
			group by 
					CLAIM_NUM 
					,INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK 
					,F1ROLL
					,F1PROL
					,F1GROL
					,F1DLOT
					,Claim_Status_Type_Cd
			having 
					sum(TOTAL_CLAIMS_DOLLAR_AMT) >0 
		) Claim ,
		( 
				select 
						INVENTORY_STYLE_CD 
						,INVENTORY_SIZE_CD 
						,INVENTORY_BACKING_CD 
						,INVENTORY_COLOR_CD 
						,DIM_DIVISION_CUSTOMER_GK 
						,sum(TOTAL_CLAIMS_DOLLAR_AMT) as Total_Claim 
						,count(distinct CLAIM_NUM) as Claim_Cnt 
				from 
						"MHK_EDW_MART"."FACT_CUSTOMER_CLAIMS" A 
						,"MHK_EDW_MART"."DIM_CLAIM_STATUS" B 
						,"MHK_EDW_MART"."DIM_CLAIM_REASON" C 
						,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" D 
						,"MHK_EDW_MART"."DIM_BUSINESS_UNIT_DATE" E 
						,"MHK_EDW_MART"."FIP010"  F
						
				where 
						A.DIM_CLAIM_STATUS_PK=B.DIM_CLAIM_STATUS_PK 
						and A.DIM_CLAIM_REASON_PK = C.DIM_CLAIM_REASON_PK 
						and A.DIM_FINISHED_PRODUCT_MFG_GK = D.DIM_FINISHED_PRODUCT_MFG_GK 
						and A.FINISHED_ROLL_NUM = F.F1ROLL
						and QUALITY_RELATED_CD = 'Y' 
						and Claim_Status_Type_Cd in ('48','49','50','51','30','35','36','34') 
						and A.CLAIM_STATUS_DATE_PK =E.DIM_DATE_PK 
						and BUSINESS_UNIT_CD='MF' 
						and FIRST_OF_FISCAL_YEAR_DT between '2016-01-01' and Current_date 
						and A.DIM_FINISHED_PRODUCT_MFG_GK >0 
						
				group by 
					INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK
				
				having sum(TOTAL_CLAIMS_DOLLAR_AMT) >0  )HClaim , 
				( 
						select 
								INVENTORY_STYLE_CD 
								,INVENTORY_SIZE_CD 
								,INVENTORY_BACKING_CD 
								,INVENTORY_COLOR_CD 
								,SOLD_TO_CUSTOMER_GK 
								,Count(distinct to_char(SOLD_TO_CUSTOMER_GK) || to_char(DOCUMENT_NUM)) as Trx_cnt 
								,sum(Net_Sales_amt) as Net_Sales 
					  from "MHK_EDW_MART"."FACT_CAMS_GL_SALES_INTERFACE" A 
					  		,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" B 
					  		,"MHK_EDW_MART"."DIM_BUSINESS_UNIT_DATE" C  
					  where 
					  		A.ENTRY_DATE_PK =C.DIM_DATE_PK 
					  		and C.BUSINESS_UNIT_CD='MF' 
					  		and FIRST_OF_FISCAL_YEAR_DT 
					  		between '2016-01-01' and Current_date 
					  		and A.FINISHED_PRODUCT_MFG_GK = B.DIM_FINISHED_PRODUCT_MFG_GK 
					  group by 
					  		INVENTORY_STYLE_CD 
					  		,INVENTORY_SIZE_CD 
					  		,INVENTORY_BACKING_CD 
					  		,INVENTORY_COLOR_CD 
					  		,SOLD_TO_CUSTOMER_GK 
					  having sum(Net_Sales_amt)>0 )HSales  
					  where 
					  		Claim.INVENTORY_STYLE_CD = HClaim.INVENTORY_STYLE_CD 
					  		and Claim.INVENTORY_SIZE_CD = HClaim.INVENTORY_SIZE_CD 
					  		and Claim.INVENTORY_BACKING_CD = HClaim.INVENTORY_BACKING_CD 
					  		and Claim.INVENTORY_COLOR_CD = HClaim.INVENTORY_COLOR_CD 
					  		and Claim.DIM_DIVISION_CUSTOMER_GK = HClaim.DIM_DIVISION_CUSTOMER_GK
					  		and	Claim.INVENTORY_STYLE_CD = HSales.INVENTORY_STYLE_CD 
					  		and Claim.INVENTORY_SIZE_CD = HSales.INVENTORY_SIZE_CD 
					  		and Claim.INVENTORY_BACKING_CD = HSales.INVENTORY_BACKING_CD 
					  		and Claim.INVENTORY_COLOR_CD = HSales.INVENTORY_COLOR_CD 
					  		and Claim.DIM_DIVISION_CUSTOMER_GK = HSales.SOLD_TO_CUSTOMER_GK 
					  		 
					 group by 
					 		Claim.CLAIM_NUM 
					 		,Claim.INVENTORY_STYLE_CD 
					 		,Claim.INVENTORY_SIZE_CD 
					 		,Claim.INVENTORY_BACKING_CD 
					 		,Claim.INVENTORY_COLOR_CD 
					 		,Claim.DIM_DIVISION_CUSTOMER_GK
					 		,Claim.F1ROLL
							,Claim.F1PROL
							,Claim.F1GROL
							,Claim.F1DLOT
							,Claim.Claim_Status_Type_Cd;
							
						
""",conn)
conn.close()
#df.head(10)
print(df.shape)
print(list(df.columns))



(29980, 16)
['CLAIM_NUM', 'INVENTORY_STYLE_CD', 'INVENTORY_SIZE_CD', 'INVENTORY_BACKING_CD', 'INVENTORY_COLOR_CD', 'DIM_DIVISION_CUSTOMER_GK', 'F1ROLL', 'F1PROL', 'F1GROL', 'F1DLOT', 'CLAIM_STATUS_TYPE_CD', 'CLAIM', 'TOTAL_CLAIM', 'TOTAL_CLAIM_CNT', 'TRX_CNT', 'NET_SALES']


In [2]:
df.head(10)



,CLAIM_NUM,INVENTORY_STYLE_CD,INVENTORY_SIZE_CD,INVENTORY_BACKING_CD,INVENTORY_COLOR_CD,DIM_DIVISION_CUSTOMER_GK,F1ROLL,F1PROL,F1GROL,F1DLOT,CLAIM_STATUS_TYPE_CD,CLAIM,TOTAL_CLAIM,TOTAL_CLAIM_CNT,TRX_CNT,NET_SALES
0,7013035,28313,1200,A,888,319858,04930866,,4720427,A35764,50,5.03,10.06,1,33,196213.37
1,7013035,28313,1200,A,888,319858,04930750,,4719730,A35764,50,5.03,10.06,1,33,196213.37
2,7043980,28072,1200,A,717,238677,05574644,05420097,4822608,P37974,50,158.98,158.98,1,218,867951.71
3,7140445,P61,0600,PB,12,322857,11662943,,,PAD,50,38.70,116.10,1,70,46242.60
4,7140445,P61,0600,PB,12,322857,11662942,,,PAD,50,38.70,116.10,1,70,46242.60
5,7140445,P61,0600,PB,12,322857,11662944,,,PAD,50,38.70,116.10,1,70,46242.60
6,6979369,27611,1200,A,747,238677,04346503,03523530,4628843,A31915,51,217.50,302.51,2,224,729542.08
7,7003369,27620,1200,A,833,238677,03654080,,4633508,A32265$,30,562.67,1071.86,3,178,678312.25
8,7063825,27898,1200,A,81289,269723,07514287,,5007007,K60219,50,328.00,328.00,1,78,521236.59
9,7137367,28274,1200,H5,720,241551,11845146,11781678,5406905,P82656,48,4096.64,4096.64,1,3,3577.32


In [3]:
df.head(10)

,CLAIM_NUM,INVENTORY_STYLE_CD,INVENTORY_SIZE_CD,INVENTORY_BACKING_CD,INVENTORY_COLOR_CD,DIM_DIVISION_CUSTOMER_GK,F1ROLL,F1PROL,F1GROL,F1DLOT,CLAIM_STATUS_TYPE_CD,CLAIM,TOTAL_CLAIM,TOTAL_CLAIM_CNT,TRX_CNT,NET_SALES
0,7013035,28313,1200,A,888,319858,04930866,,4720427,A35764,50,5.03,10.06,1,33,196213.37
1,7013035,28313,1200,A,888,319858,04930750,,4719730,A35764,50,5.03,10.06,1,33,196213.37
2,7043980,28072,1200,A,717,238677,05574644,05420097,4822608,P37974,50,158.98,158.98,1,218,867951.71
3,7140445,P61,0600,PB,12,322857,11662943,,,PAD,50,38.70,116.10,1,70,46242.60
4,7140445,P61,0600,PB,12,322857,11662942,,,PAD,50,38.70,116.10,1,70,46242.60
5,7140445,P61,0600,PB,12,322857,11662944,,,PAD,50,38.70,116.10,1,70,46242.60
6,6979369,27611,1200,A,747,238677,04346503,03523530,4628843,A31915,51,217.50,302.51,2,224,729542.08
7,7003369,27620,1200,A,833,238677,03654080,,4633508,A32265$,30,562.67,1071.86,3,178,678312.25
8,7063825,27898,1200,A,81289,269723,07514287,,5007007,K60219,50,328.00,328.00,1,78,521236.59
9,7137367,28274,1200,H5,720,241551,11845146,11781678,5406905,P82656,48,4096.64,4096.64,1,3,3577.32
